# 2.데이터 로드 및 패키지 임포트(Loading the data and python packages)

## 2.1. 라이브러리 임포트

In [ ]:
# 라이브러리 임포트 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import statsmodels.api as sm1
from sklearn.model_selection import TimeSeriesSplit



## 2.2. 데이터 로드
- ‘업비트’에서 거래된 ‘이더리움’(1분봉) 1년치 거래 데이터
  - 날짜(Date) : 2021년 3월 22일 12시 00분 ~ 2022년 3월 24일 00시 18분 (UTC 기준)
  - 시가(Open) : 처음 거래된 가격
  - 고가(High)  : 가장 높은 금액으로 거래된 가격
  - 저가(Low) : 가장 낮은 금액으로 거래된 가격
  - 종가(Close) : 맨 마지막으로 거래된 가격
  - 거래량(Volume) :  매도자 및 매수자 거래를 체결할 때 발생 하는 거래의 양



In [ ]:
# data path 설정

data_path = '/content/drive/MyDrive/Aiffel Hackathon/colab_dataset/1. upbit_ETH(raw data).csv'


In [ ]:
# 데이터셋 로드

dataset = pd.read_csv(data_path)

dataset.head()

df = dataset

In [ ]:
# 경고 메세지 사용 안하기

import warnings
warnings.filterwarnings


##4.1. Data Cleaning

- 결측치 전처리 - raw data에서 직접 전처리
  - (결측치) 업비트 거래소 점검에 따라 약 30시간의 거래 기록(OHLCV) 결측치 발생
  - (전처리) 다른 국내 거래소(빗썸, 코빗) 거래 기록 중 업비트와 가장 유사한 내역으로 대체
    - 가상화폐는 주식 시장과 달리 민간 거래소(대표 4곳)에서 거래가 체결됨
    - 동일한 가상화폐도 거래소마다 가격이 상이하지만, 차익거래로 인해 비슷한 가격대로 수렴하는 경향을 보임


In [ ]:
# 결측치 확인

print('Null Values =', dataset.isnull().values.any())  


## 4.2. 데이터 라벨링 - 트랜드 스캐닝
1. 라벨링의 특수성
  - 개와 고양이를 분류하는 모델의 경우, 라벨링 기준이 비교적 명확함
가격 추이를 라벨링하는 경우, 그 기준이 사람마다 상이할 수 있음
  - 동일 시점의 가격도 투자자의 패턴에 따라 상승 및 하락 여부 판단이 다름
      - ex) 매수 후 6시간이 지난 시점에 가격이 10% 상승했다고 가정할 경우, 단기 투자자는 매도 시점으로 판단 할 수 있지만 장기 투자자는 보유 신호로 해석할 수 있음

2. 데이터 스무싱
  - 분(minute) 단위로 가격이 변하는 데이터 특성상 노이즈가 심함
  - 중앙 이동평균값(Centered Moving Average)을 활용하여 종가 보정

3. 라벨링 - 트랜드 스캐닝(Trend-Scanning)
  - (t-value 계산) 한 시점에서 60분 간격(step)으로 360분까지 t-value 값 계산
    - t-value > 0: 양의 상관관계 - >  상승
    - t-value < 0 : 음의상관 관계 - > 하락
  - (트랜드 스캐닝) 
    - 라벨링 결과
      - 라벨링 비율
        - (상승) 36%
        - (하락) 33%
        - (보합) 31%
    - 추세 반전( 상승 -> 하락 or 하락 -> 상승)
      - 하루 1회 이상 추세 반전 확인



In [ ]:
# Centered MA 함수 생성

def CMA(df_close, n):
    CMA = df_close.rolling(window=n, center=True).mean()
    return CMA


# 10분/30분/60분/120분/180분/240분/360분/420분/480분/540분/600분/720분  CMA 생성
dataset['CMA_10'] = CMA(dataset['Close'], 10)
dataset['CMA_30'] = CMA(dataset['Close'], 30)
dataset['CMA_60'] = CMA(dataset['Close'], 60)
dataset['CMA_120'] = CMA(dataset['Close'], 120)
dataset['CMA_180'] = CMA(dataset['Close'], 360)
dataset['CMA_240'] = CMA(dataset['Close'], 240)
dataset['CMA_360'] = CMA(dataset['Close'], 360)
dataset['CMA_420'] = CMA(dataset['Close'], 420)
dataset['CMA_480'] = CMA(dataset['Close'], 480)
dataset['CMA_540'] = CMA(dataset['Close'], 540)
dataset['CMA_600'] = CMA(dataset['Close'], 600)
dataset['CMA_720'] = CMA(dataset['Close'], 720)
dataset.head(100)





In [ ]:
# 종가 및 CMA_360 시각화



fig = plt.figure(figsize=(15, 7))

plt.plot(dataset['Close'], linewidth=5)

plt.plot(dataset['CMA_360'], linewidth=3)

plt.show()



In [ ]:
# 1일치 주가 및 CMA_360 시각화


plt.plot(dataset['Close'][:1440], label='Close')
plt.plot(dataset['CMA_360'][:1440], label='CMA_360')
plt.grid()
plt.rcParams['figure.figsize'] = (80, 60)
plt.show()



In [ ]:
# 라벨링에 맞게 데이터셋(dataset) 변형


pd.to_datetime(dataset['Date'])

dataset.set_index('Date', inplace=True)

dataset.head()


In [ ]:
# t-Value 생성 함수

def tValLinR(close):
    """tValue from a linear trend via SNIPPET 5.1 T-VALUE OF A LINEAR TREND
    Args:
        close (Series): close prices to search through
    Returns:
        float: t-value
    Example:
        >>> lookforward = 5
        >>> start = 0
        >>> stop = stop + lookforward
        >>> df1 = close.iloc[start:stop]
        >>> tValLinR(df1.values)
    """
    x = np.ones((close.shape[0], 2))
    x[:, 1] = np.arange(close.shape[0])
    ols = sm1.OLS(close, x).fit()
    return ols.tvalues[1]


In [ ]:
# Trend Scanning 함수 생성

def getBinsFromTrend(molecule, close, span):
    """Derive labels from the sign of t-value of linear trend via SNIPPET 5.2
    IMPLEMENTATION OF THE TREND-SCANNING METHOD
    Args:
        molecule (DatetimeIndex): start times of the event
        close (Series): close prices of your large df
        span (args for range): list: [start, stop, step]
    Returns:
        pd.DataFrame: trends
            - t1: End time for the identified trend
            - tVal: t-value associated with the estimated trend coefficient
            - bin: Sign of the trend
    Example:
        >>> df["mavg"] = df.close.rolling(10).mean()
        >>> df["entry"] = df[(df.close.shift(1) < df.mavg) & (df.close > df.mavg)].close
        >>> getBinsFromTrend(molecule=df["entry"].dropna().index, close=df.close, span=[5, 20, 5])
    """

    out = pd.DataFrame(index=molecule, columns=["t1", "tVal", "bin","max_day"])
    hrzns = range(*span)

    for dt0 in molecule:
        df0 = pd.Series()
        iloc0 = close.index.get_loc(dt0)
        if iloc0 + max(hrzns) >= close.shape[0]:            
            continue

        for hrzn in hrzns:
            # dt1 -> dt1(dt0+hrzn)의 인덱스
            dt1 = close.index[iloc0 + hrzn ] #코드 수정: iloc0 + hrzn -1 에서 -1삭제 , 위 if문의 부등호를 조절함
            df1 = close.loc[dt0:dt1]    # dt0 ~ dt1(dt0+hrzn) 행 조회
            df0.loc[dt1] = tValLinR(df1.values) #df0데이터 프레임의 dt1 column에 tval값이 들어감
            
            
            

        dt1 = df0.replace([-np.inf, np.inf, np.nan], 0).abs().idxmax() #최대값의 인덱스를 dt1에 저장함
        out.loc[dt0, ["t1", "tVal", "bin","max_day"]] = (
            df0.index[-1],#회귀선을 그은 마지막 시간
            df0[dt1],#df0 테이블에는 tval값이 각 분마다 저장되어있고, dt1에는 최댓값의 인덱스 > 따라서 df0[dt1]에는 최댓값이 들어감
            np.sign(df0[dt1]),
            dt1
        )  # prevent leakage

    out["t1"] = pd.to_datetime(out["t1"])
    out["bin"] = pd.to_numeric(out["bin"], downcast="signed")

    return out.dropna(subset=["bin"])
  
  

In [ ]:
# Trand Scanning 함수 실행


trend = getBinsFromTrend(
    molecule = dataset["CMA_360"].dropna().index, 
    close = dataset["CMA_360"].dropna(), 
    span=[60,361,60],
)
trend



In [ ]:
2# t-value의 평균 및 표준편차 


tval_mean = abs(trend['tVal']).mean()
tval_std = abs(trend['tVal']).std()



print(f"tval_mean:{tval_mean},tval_std:{tval_std},tval_max:{abs(trend['tVal']).max()}")



In [ ]:
# t-value < 80 행 추출 및 보합(0)으로 라벨링

trend.loc[abs(trend['tVal']) < 80]

trend.loc[abs(trend['tVal']) < 80, 'bin'] = 0


In [ ]:
# 라벨값(상승,하락,보합) 별 컬럼 추가

trend['up'] = (trend['bin'] == 1)
trend['down'] = (trend['bin'] == -1)
trend['hold'] = (trend['bin']==0)


In [ ]:
# 라벨링 리스트 생성 

  # 상승
up_list = []
prev_val = False

for inx, val in trend['up'].iteritems():
    if prev_val != val:
        if val:
            start = inx
        else:
            up_list.append((start, inx))

    prev_inx = inx
    prev_val = val


  # 하락
down_list = []
prev_val = False

for inx, val in trend['down'].iteritems():
    if prev_val != val:
        if val:
            start = inx
        else:
            down_list.append((start, inx))

    prev_inx = inx
    prev_val = val


  # 보합
hold_list = []
prev_val = False

for inx, val in trend['hold'].iteritems():
    if prev_val != val:
        if val:
            start = inx
        else:
            hold_list.append((start, inx))

    prev_inx = inx
    prev_val = val



    


In [ ]:
# 라벨링 데이터 시각화


plt.plot(dataset['CMA_360'][:10000],linewidth = 10)
plt.plot(dataset['Close'][:10000],linewidth = 5)
for (up_start, up_end) in up_list:
    plt.axvspan(up_start, up_end, color='red', alpha=0.3)
for (down_start, down_end) in down_list:
    plt.axvspan(down_start, down_end, color='blue', alpha=0.3)
for (hold_start, hold_end) in hold_list:
    plt.axvspan(hold_start, hold_end, color='green', alpha=0.3)



In [ ]:
# 타겟 컬럼 값의 개수 출력


trend['bin'].value_counts()


In [ ]:
# 데이터프레임에 타겟('signal') 컬럼 결합

dataset['signal'] = trend['bin']

dataset.tail(1000)


In [ ]:
# centered MA 컬럼 제거

dataset=dataset.drop(['CMA_10', 'CMA_30', 
                      'CMA_60', 'CMA_120', 'CMA_180', 'CMA_240', 'CMA_360', 'CMA_420', 
                      'CMA_480', 'CMA_540', 'CMA_600', 'CMA_720'], axis=1)


In [ ]:
# 데이터프레임 인덱스 리셋


dataset.reset_index(drop=False, inplace=True)

dataset.head(1000)
